# Parse tables to database

https://nbviewer.jupyter.org/github/FinanceData/OpenDartReader/blob/master/docs/OpenDartReader_reference_manual.ipynb


```
KOSPI
└── Companys
    └── Receipts
        ├── BS: Financial statement(Balance Sheet)
        ├── IS: Income Statement
        ├── CIS: Comprehensive Income Statement
        ├── CF: Cash flow statement
        └── SCE: Statement of Changes in Equity
```

<p align="center">
    <img alt="Alt Text" src="https://g.gravizo.com/svg?digraph%20G%20%7B%0A%20%201%20%5Blabel%3D%22KOSPI%22%2C%20fontcolor%3Dblack%2C%20shape%3Dbox%5D%3B%0A%20%202%20%5Blabel%3D%22Company1%22%2C%20fontcolor%3Dblack%2C%20shape%3Dbox%5D%3B%0A%20%203%20%5Blabel%3D%22Company2%22%2C%20fontcolor%3Dblack%2C%20shape%3Dbox%5D%3B%0A%20%204%20%5Blabel%3D%22...%22%2C%20fontcolor%3Dblack%2C%20shape%3Dbox%5D%3B%0A%20%205%20%5Blabel%3D%22Receipt%20No.1%22%2C%20fontcolor%3Dblack%2C%20shape%3Dbox%5D%3B%0A%20%206%20%5Blabel%3D%22Receipt%20No.2%22%2C%20fontcolor%3Dblack%2C%20shape%3Dbox%5D%3B%20%20%20%20%0A%20%207%20%5Blabel%3D%22...%22%2C%20fontcolor%3Dblack%2C%20shape%3Dbox%5D%3B%0A%20%208%20%5Blabel%3D%22BS%22%2C%20fontcolor%3Dblack%2C%20shape%3Dbox%5D%3B%0A%20%209%20%5Blabel%3D%22IS%22%2C%20fontcolor%3Dblack%2C%20shape%3Dbox%5D%3B%0A%20%2010%20%5Blabel%3D%22CIS%22%2C%20fontcolor%3Dblack%2C%20shape%3Dbox%5D%3B%0A%20%2011%20%5Blabel%3D%22CF%22%2C%20fontcolor%3Dblack%2C%20shape%3Dbox%5D%3B%0A%20%2012%20%5Blabel%3D%22SCE%22%2C%20fontcolor%3Dblack%2C%20shape%3Dbox%5D%3B%0A%20%201%20-%3E%202%20-%3E%205%3B%0A%20%201%20-%3E%203%3B%0A%20%201%20-%3E%204%3B%0A%20%202%20-%3E%206%3B%0A%20%202%20-%3E%207%3B%0A%20%206%20-%3E%208%3B%0A%20%206%20-%3E%209%3B%0A%20%206%20-%3E%2010%3B%0A%20%206%20-%3E%2011%3B%0A%20%206%20-%3E%2012%3B%0A%7D" />
</p>
<details>
<summary>How to create graph in markdown?</summary>

```python
from urllib.parse import quote
raw = """digraph G {
  1 [label="KOSPI", fontcolor=black, shape=box];
  2 [label="Company1", fontcolor=black, shape=box];
  3 [label="Company2", fontcolor=black, shape=box];
  4 [label="...", fontcolor=black, shape=box];
  5 [label="Receipt No.1", fontcolor=black, shape=box];
  6 [label="Receipt No.2", fontcolor=black, shape=box];    
  7 [label="...", fontcolor=black, shape=box];
  8 [label="BS", fontcolor=black, shape=box];
  9 [label="IS", fontcolor=black, shape=box];
  10 [label="CIS", fontcolor=black, shape=box];
  11 [label="CF", fontcolor=black, shape=box];
  12 [label="SCE", fontcolor=black, shape=box];
  1 -> 2 -> 5;
  1 -> 3;
  1 -> 4;
  2 -> 6;
  2 -> 7;
  6 -> 8;
  6 -> 9;
  6 -> 10;
  6 -> 11;
  6 -> 12;
}"""
txt = quote(raw)
```
    
copy the text behind https://g.gravizo.com/svg?
</details>

# Finance DataReader

In [1]:
from private.apikey import APIKEY

import pandas as pd
import OpenDartReader
import FinanceDataReader as fdr

dart = OpenDartReader(APIKEY) 

In [2]:
stocks = fdr.StockListing("KOSPI")
stocks = stocks.loc[~stocks["Sector"].isnull(), :]
stocks_syms = stocks["Symbol"].values
stocks = stocks.reset_index(drop=True)

In [7]:
for c in stocks.columns:
    print(c, stocks[c].dtype)

Symbol object
Market object
Name object
Sector object
Industry object
ListingDate datetime64[ns]
SettleMonth object
Representative object
HomePage object
Region object


In [8]:
stocks["ListingDate"] = stocks["ListingDate"].astype(str)

In [11]:
import sqlite3
from pathlib import Path

db_path = Path("./private/")
conn = sqlite3.connect(db_path / "kospi.db")
# Samsung Electronic: 005930
# conn = sqlite3.connect(db_path / "samsung_new.db")
c = conn.cursor()

In [12]:
stocks.to_sql("kospi", conn, index=True)

## Company Basic

- corp_name: 정식명칭
- corp_name_eng: 영문명칭
- stock_name: 종목명 또는 약식명칭 
- stock_code: 상장회사인 경우 주식의 종목코드
- ceo_nm: 대표자명
- crop_cls: 법인구분
- jurir_no: 법인등록번호
- bizr_no: 사업자등록번호
- adres: 주소
- hm_url: 홈페이지
- ir_url: IR홈페이지
- phn_no: 전화번호
- fax_no: 팩스번호
- induty_code: 업종코드
- estdt: 설립일
- acc_mt: 결산월

In [13]:
from tqdm.notebook import tqdm

In [14]:
basic_cols = [
    'corp_name', 'corp_name_eng', 'stock_name', 'stock_code', 'ceo_nm', 'corp_cls', 
    'jurir_no', 'bizr_no', 'adres', 'hm_url', 'ir_url', 'phn_no', 'fax_no', 'induty_code', 'est_dt', 'acc_mt'
]
df_basic = pd.DataFrame([dart.company(s) for s in tqdm(stocks_syms)]).loc[:, basic_cols]
df_basic.to_sql("company", conn, index=True)

  0%|          | 0/810 [00:00<?, ?it/s]

In [15]:
for col in df_basic.columns:
    print(col, df_basic[col].dtype)

corp_name object
corp_name_eng object
stock_name object
stock_code object
ceo_nm object
corp_cls object
jurir_no object
bizr_no object
adres object
hm_url object
ir_url object
phn_no object
fax_no object
induty_code object
est_dt object
acc_mt object


In [14]:
# df_samsung = pd.DataFrame([dart.company("005930")]).loc[:, basic_cols]

# DROP TABLE if exists
# sql = "DROP TABLE company"
# res = c.execute(sql)

# insert into "company" table
# df_samsung.to_sql("company", conn, index=True)

In [18]:
sql = "SELECT * FROM company"
res = c.execute(sql)
col_company = list(map(lambda x: x[0], res.description))
for col, value in zip(col_company, res.fetchone()):
    print(f"{col}: {value}")

index: 0
corp_name: AJ네트웍스 주식회사
corp_name_eng: AJ Networks Co.,Ltd.
stock_name: AJ네트웍스
stock_code: 095570
ceo_nm: 박대현
corp_cls: Y
jurir_no: 1101111874654
bizr_no: 2148648586
adres: 서울특별시 송파구 정의로8길 9 (문정동,AJ빌딩)
hm_url: www.ajnet.co.kr
ir_url: 
phn_no: 02-6363-9999
fax_no: 02-6240-0888
induty_code: 76320
est_dt: 20000210
acc_mt: 12


## Report

- corp (문자열): 검색대상 회사의 종목코드를 지정합니다. 고유번호, 회사이름도 가능합니다.
- key_word (문자열): 조회 내용 지정, 아래 "key_word 항목"을 참고하십시오 ('증자','배당','자기주식','최대주주','최대주주변동','소액주주','임원','직원','임원개인보수','임원전체보수','개인별보수','타법인출자')
- bsns_year (문자열 혹은 정수값): 사업연도
- reprt_code (문자열): 보고서 코드 ('11013'=1분기보고서, '11012'=반기보고서, '11014'=3분기보고서, '11011'=사업보고서)

반환값 (DataFrame): 조회 결과를 데이터프레임(DataFrame)으로 반환합니다. 데이터프레임의 각 컬럼은 다음과 같습니다.

- rcept_no: 접수번호
- corp_cls: 법인구분 Y(유가), K(코스닥), N(코넥스), E(기타)
- corp_code: 고유번호
- corp_name: 법인명

key_word 항목 지정에 따라 결과 데이터의 컬럼이 달라집니다. '배당' - 배당에 관한 사항

- se: 구분. 유상증자(주주배정), 전환권행사 등
- stock_knd: 주식 종류
- thstrm: 당기
- frmtrm: 전기
- lwfr: 전전기

In [19]:
corp = "005930"
bsns_year = 2020

In [20]:
# ['증자', '배당', '자기주식', '최대주주', '최대주주변동', '소액주주', '임원', '직원', '임원개인보수', '임원전체보수', '개인별보수', '타법인출자']
dart.report(corp, '개인별보수', bsns_year, reprt_code='11011')

,rcept_no,corp_cls,corp_code,corp_name,nm,ofcps,mendng_totamt,mendng_totamt_ct_incls_mendng
0,20210309000744,Y,00126380,삼성전자,권오현,고 문,"17,233,000,000",-
1,20210309000744,Y,00126380,삼성전자,윤부근,고 문,"11,527,000,000",-
2,20210309000744,Y,00126380,삼성전자,신종균,고 문,"11,327,000,000",-
3,20210309000744,Y,00126380,삼성전자,전동수,前고문,"10,908,000,000",-
4,20210309000744,Y,00126380,삼성전자,김기남,대표이사,"8,274,000,000",-


## Finstate
- corp (문자열): 검색대상 회사의 종목코드를 지정합니다. 고유번호, 회사이름도 가능합니다.
- bsns_year (문자열 혹은 정수값): 사업연도
- reprt_code (문자열): 보고서 코드 ('11013'=1분기보고서, '11012'=반기보고서, '11014'=3분기보고서, '11011'=사업보고서)

반환값 (DataFrame): 조회 결과를 데이터프레임(DataFrame)으로 반환합니다. 데이터프레임의 각 컬럼은 다음과 같습니다.

- rcept_no: 접수번호
- corp_code: 사업 연도
- stock_code: 종목 코드
- reprt_code: 보고서 코드
- account_nm: 계정명 (예: 자본총계)
- fs_div: 개별/연결구분 ('CFS'=연결재무제표, 'OFS'=재무제표)
- fs_nm: 개별/연결명 ('연결재무제표' 또는 '재무제표')
- sj_div: 재무제표구분 ('BS'=재무상태표, 'IS'=손익계산서)
- sj_nm: 재무제표명 ( '재무상태표' 또는 '손익계산서')
- thstrm_nm: 당기명
- thstrm_dt: 당기일자
- thstrm_amount: 당기금액
- thstrm_add_amount: 당기누적금액
- frmtrm_nm: 전기명
- frmtrm_dt: 전기일자
- frmtrm_amount: 전기금액
- frmtrm_add_amount: 전기누적금액
- bfefrmtrm_nm: 전전기명
- bfefrmtrm_dt: 전전일자
- bfefrmtrm_amount: 전전기금액
- ord: 계정과목 정렬순서

In [21]:
df_summary = dart.finstate(corp, bsns_year, reprt_code="11011")
for col in ["thstrm_amount", "frmtrm_amount", "bfefrmtrm_amount"]:
    df_summary.loc[:, col] = df_summary.loc[:, col].apply(lambda x: int("".join(x.split(","))))
    
cols = ["fs_div", "sj_div", "account_nm", "thstrm_nm", "thstrm_amount"] # "frmtrm_nm", "frmtrm_amount", "bfefrmtrm_nm", "bfefrmtrm_amount"]
df_summary.loc[:, cols].groupby(["thstrm_nm", "fs_div", "sj_div", "account_nm"]).agg("sum")

thstrm_amount
thstrm_nm fs_div sj_div account_nm                 
제 52 기    CFS    BS     부채총계        102287702000000
                        비유동부채        26683351000000
                        비유동자산       180020139000000
                        유동부채         75604351000000
                        유동자산        198215579000000
                        이익잉여금       271068211000000
                        자본금            897514000000
                        자본총계        275948016000000
                        자산총계        378235718000000
                 IS     당기순이익        26407832000000
                        매출액         236806988000000
                        법인세차감전 순이익   36345117000000
                        영업이익         35993876000000
          OFS    BS     부채총계         46347703000000
                        비유동부채         1934799000000
                        비유동자산       155865878000000
                        유동부채         44412904000000
                        유동자산         73798549000000
                        이익잉여금       178284102000000
                        자본금            897514000000
                        자본총계        183316724000000
                        자산총계        229664427000000
                 IS     당기순이익        15615018000000
                        매출액         166311191000000
                        법인세차감전 순이익   20451923000000
                        영업이익         20518974000000

In [22]:
df_summary["account_nm"].unique()

array(['유동자산', '비유동자산', '자산총계', '유동부채', '비유동부채', '부채총계', '자본금', '이익잉여금',
       '자본총계', '매출액', '영업이익', '법인세차감전 순이익', '당기순이익'], dtype=object)

In [23]:
df_summary.loc[:, ["fs_div", "fs_nm"]].loc[~df_summary.loc[:, ["fs_div", "fs_nm"]].duplicated(keep="first")]

,fs_div,fs_nm
0,CFS,연결재무제표
13,OFS,재무제표


## Query 와 SQL 만들기

- [시간]: BS: '제\*\*기', '20\*\*년도', '올해', '작년' / IS: '20\*\*년도 부터 20\*\*년 까지', 
- [계정]: BS: '유동자산', '비유동자산', '자산총계', '유동부채', '비유동부채', '부채총계', '자본금', '이익잉여금', '자본총계' / IS: '매출액', '영업이익', '법인세차감전 순이익', '당기순이익'
- [질문]: '얼마나돼', '얼마야', '어떻게돼'

**format의 형태**

틀린 질문을 만들수 없게 로직짜기

단순 계정 관련 질문:
- Q: 삼성전자 [시간]의 [계정]은/는 [질문]? 
- Q: [시간]에서 삼성전자의 [계정]은/는 [질문]?

## XBRL 표준계정과목체계(계정과목)

In [24]:
df_bs1 = dart.xbrl_taxonomy(sj_div="BS1")
df_is1 = dart.xbrl_taxonomy(sj_div="IS1")
bs1_dict = {k: v for k, v in df_bs1.loc[:, ["label_kor", "account_nm"]].values}
is1_dict = {k: v for k, v in df_is1.loc[:, ["label_kor", "account_nm"]].values}

In [25]:
for x, v in list(is1_dict.items()):
    if "당기순이익" in x:
        print(x, v)
        break

당기순이익(손실) ProfitLoss


In [28]:
account_dict = {
    "유동자산" : "CurrentAssets",
    "비유동자산" : "NoncurrentAssets",
    "자산총계" : "Assets",
    "유동부채" : "CurrentLiabilities",
    "비유동부채" : "NoncurrentLiabilities",
    "부채총계" : "Liabilities",
    "자본금" : "IssuedCapital",
    "이익잉여금" : "RetainedEarnings",
    "자본총계" : "Equity",
    "매출액" : "Revenue",
    "영업이익" : "OperatingIncomeLoss",
    "법인세차감전 순이익" : "ProfitLossBeforeTax",
    "당기순이익" : "ProfitLoss"
}

In [29]:
for sj, acc in df_summary.loc[df_summary["fs_div"] == "CFS", ["sj_div", "account_nm"]].values:
    print(f"{acc} = {account_dict.get(acc)}")

유동자산 = CurrentAssets
비유동자산 = NoncurrentAssets
자산총계 = Assets
유동부채 = CurrentLiabilities
비유동부채 = NoncurrentLiabilities
부채총계 = Liabilities
자본금 = IssuedCapital
이익잉여금 = RetainedEarnings
자본총계 = Equity
매출액 = Revenue
영업이익 = OperatingIncomeLoss
법인세차감전 순이익 = ProfitLossBeforeTax
당기순이익 = ProfitLoss


In [26]:
def preprocess_finstate(df):
#     account_dict = {
#         "유동자산" : "CurrentAssets",
#         "비유동자산" : "NoncurrentAssets",
#         "자산총계" : "Assets",
#         "유동부채" : "CurrentLiabilities",
#         "비유동부채" : "NoncurrentLiabilities",
#         "부채총계" : "Liabilities",
#         "자본금" : "IssuedCapital",
#         "이익잉여금" : "RetainedEarnings",
#         "자본총계" : "Equity",
#         "매출액" : "Revenue",
#         "영업이익" : "OperatingIncomeLoss",
#         "법인세차감전 순이익" : "ProfitLossBeforeTax",
#         "당기순이익" : "ProfitLoss"
#     }
#     df.loc[:, "account_nm"] = df.loc[:, "account_nm"].apply(account_dict.get)

    for col in ["thstrm_amount", "frmtrm_amount", "bfefrmtrm_amount"]:
        if col in df.columns:
            df.loc[:, col] = df.loc[:, col].apply(lambda x: int("".join(x.split(","))))
            
    return df

In [33]:
corp = "005930"
bsns_years = list(range(2015, 2021))

In [37]:
df_receipts = pd.concat([preprocess_finstate(dart.finstate(corp, bsns_year, reprt_code="11011")).iloc[:12, :-1] for bsns_year in bsns_years]).reset_index(drop=True)
df_receipts.loc[:, "bsns_year"] = df_receipts.loc[:, "bsns_year"].astype(int)

## receipts all

당기 만 저장

In [28]:
import time

In [29]:
select_columns = ['rcept_no', 'reprt_code', 'bsns_year', 'corp_code', 'stock_code', 'fs_div', 'fs_nm', 'sj_div', 'sj_nm', 'account_nm', 'thstrm_nm','thstrm_dt', 'thstrm_amount']
bsns_years = list(range(2018, 2021))
dfs = {}
for corp in tqdm(stocks_syms):
    dfs[corp] = []
    for bsns_year in bsns_years:
        df = dart.finstate(corp, bsns_year)
        time.sleep(1)
        if df is None:
            continue
        else:
            df = df.loc[:12, select_columns]
            df.loc[df["thstrm_amount"] == "-", "thstrm_amount"] = "0"
            df.loc[:, "thstrm_amount"] = df.loc[:, "thstrm_amount"].apply(lambda x: int("".join(x.split(",")))).astype(int)
            df.loc[:, "bsns_year"] = df.loc[:, "bsns_year"].astype(int)
            dfs[corp].append(df)

  0%|          | 0/810 [00:00<?, ?it/s]

In [30]:
# 59 companys don's have receipts for now
sum([True if len(dfs[corp]) == 0 else False for corp in tqdm(stocks_syms)])

  0%|          | 0/810 [00:00<?, ?it/s]

60

In [31]:
# save to sql
for corp in tqdm(stocks_syms):
    if len(dfs[corp]) == 0:
        continue
    df = pd.concat(dfs[corp]).reset_index(drop=True)
    table_name = f"{corp}"
    df.to_sql(table_name, conn, index=True)

  0%|          | 0/810 [00:00<?, ?it/s]